In [ ]:
import numpy as np
import math
import matplotlib.pyplot as plt
from tensorflow.keras.datasets import cifar10
from tensorflow.keras.layers import Conv2D, ZeroPadding2D,BatchNormalization, Activation, MaxPooling2D, Add
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense, Dropout
from tensorflow.keras import models
from tensorflow.keras import optimizers
from tensorflow.keras import utils
from tensorflow.keras import Input




#*****************************************************
#               Standard
#*****************************************************


name_list = ('airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
num_classes = 10
(x_train, y_train), (x_test, y_test) = cifar10.load_data()
print('x_train shape:', x_train.shape)
print(x_train.shape[0], 'train samples')
print(x_test.shape[0], 'test samples')


x_train = x_train.astype('float32')
x_test = x_test.astype('float32')
x_train /= 255
x_test /= 255

# Convert class vectors to binary class matrices.
y_train = utils.to_categorical(y_train, num_classes)
y_test = utils.to_categorical(y_test, num_classes)


(datanum, h, w, channum) = x_train.shape
(_, outputdim) = y_train.shape



#Visualizing CIFAR 10
fig = plt.figure()
ims = np.random.randint(datanum, size=15)

for i in range(15):
    subplot = fig.add_subplot(3,5, i+1)
    subplot.set_xticks([])
    subplot.set_yticks([])
    subplot.set_title("%s" %name_list[np.argmax(y_train[ims[i]])])
    subplot.imshow(x_train[ims[i],:,:,:])

plt.show()




In [ ]:
# Training Parameters
epochs = 10
batch_size = 64


# *************************************************************
#               Model building
# *************************************************************


def build_network():


    def conv_block(input_tensor, filters, strides):

        filters1, filters2, filters3 = filters

        shortcut = input_tensor

        x = Conv2D(filters1, (1, 1), strides=strides, padding='valid')(input_tensor)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)

        x = Conv2D(filters2, (3, 3), strides=(1, 1), padding='same')(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)

        x = Conv2D(filters3, (1, 1), strides=(1, 1), padding='valid')(x)
        x = BatchNormalization()(x)
        shortcut = Conv2D(filters3, (1, 1), strides=strides, padding='valid')(shortcut)
        shortcut = BatchNormalization()(shortcut)

        x = Add()([x, shortcut])
        x = Activation('relu')(x)

        return x



    def identity_block(input_tensor, filters):

        filters1, filters2, filters3 = filters

        shortcut = input_tensor

        x = Conv2D(filters1, (1, 1), strides=(1, 1), padding='valid')(input_tensor)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)

        x = Conv2D(filters2, (3, 3), strides=(1, 1), padding='same')(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)

        x = Conv2D(filters3, (1, 1), strides=(1, 1), padding='valid')(x)
        x = BatchNormalization()(x)

        x = Add()([x, shortcut])
        x = Activation('relu')(x)

        return x




    #input_tensor = Input(shape=(224, 224, 3), dtype='float32', name='input')
    input_tensor = Input(shape=(32, 32, 3), dtype='float32', name='input')

    def conv1_layer(x):
        x = ZeroPadding2D(padding=(3, 3))(x)
        x = Conv2D(64, (7, 7), strides=(2, 2))(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)

        return x



    def conv2_layer(x):
        x = ZeroPadding2D(padding=(1, 1))(x)
        x = MaxPooling2D((3, 3), 2)(x)

        x = conv_block(x, filters = [64,64,256], strides=(1, 1))
        x = identity_block(x, filters = [64,64,256])
        x = identity_block(x, filters = [64,64,256])

        return x




    def conv3_layer(x):

        x = conv_block(x, filters = [128,128,512], strides=(2, 2))
        x = identity_block(x, filters = [128,128,512])
        x = identity_block(x, filters = [128,128,512])
        x = identity_block(x, filters = [128,128,512])

        return x



    def conv4_layer(x):

        x = conv_block(x, filters = [256,256,1024], strides=(2, 2))
        x = identity_block(x, filters = [256,256,1024])
        x = identity_block(x, filters = [256,256,1024])
        x = identity_block(x, filters = [256,256,1024])
        x = identity_block(x, filters = [256,256,1024])
        x = identity_block(x, filters = [256,256,1024])

        return x



    def conv5_layer(x):

        x = conv_block(x, filters = [512,512,2048], strides=(2, 2))
        x = identity_block(x, filters = [512,512,2048])
        x = identity_block(x, filters = [512,512,2048])

        return x


    x = conv1_layer(input_tensor)
    x = conv2_layer(x)
    x = conv3_layer(x)
    x = conv4_layer(x)
    x = conv5_layer(x)

    x = GlobalAveragePooling2D()(x)

    x = Dense(100, activation='relu') (x)
    x = Dropout(0.2) (x)

    output_tensor = Dense(num_classes, activation='softmax')(x)

    resnet50 = models.Model(input_tensor, output_tensor)
    resnet50.summary()

    return resnet50


model = build_network()

model.compile(optimizer=optimizers.Adam(lr=0.0001),
              loss='categorical_crossentropy',
              metrics=['accuracy'])

# Fit the model on the batches generated by datagen.flow().
history =  model.fit(x=x_train, y=y_train,batch_size=batch_size, epochs=epochs, validation_data=(x_test, y_test),
                            shuffle=True, verbose=2, workers=1)

history_dict = history.history
history_dict.keys()


In [ ]:
# *************************************************************
#               Visualization
# *************************************************************

# Training loss
loss = history.history['loss']
val_loss = history.history['val_loss']
epochs = range(1, len(loss) + 1)
plt.plot(epochs, loss, 'bo', label='Training loss')
plt.plot(epochs, val_loss, 'b', label='Validation loss')
plt.title('Training and validation loss')
plt.xlabel('Epochs')
plt.ylabel('Loss')
plt.legend()
plt.show()



# Training accuracy
plt.clf()   # 그래프를 초기화합니다
acc = history.history['accuracy']
val_acc = history.history['val_accuracy']
plt.plot(epochs, acc, 'bo', label='Training acc')
plt.plot(epochs, val_acc, 'b', label='Validation acc')
plt.title('Training and validation accuracy')
plt.xlabel('Epochs')
plt.ylabel('Accuracy')
plt.legend()
plt.show()

In [ ]:
import sklearn.metrics as skl


def plot_confusion_matrix(cm, classes,normalize=False,title='Confusion matrix',cmap=plt.cm.Blues):
    """
    This function prints and plots the confusion matrix.
    Normalization can be applied by setting `normalize=True`.
    """
    if normalize:
        cm = cm.astype('float') / cm.sum(axis=1)[:, np.newaxis]
        print("Normalized confusion matrix")
    else:
        print('Confusion matrix, without normalization')

    #print(cm)

    plt.imshow(cm, interpolation='nearest', cmap=cmap)
    plt.title(title)
    plt.colorbar()
    tick_marks = np.arange(len(classes))
    plt.xticks(tick_marks, classes, rotation=45)
    plt.yticks(tick_marks, classes)

    fmt = '.2f' if normalize else 'd'
    thresh = cm.max() / 2.

    for i in range(cm.shape[0]):
        for j in range(cm.shape[1]):
            plt.text(j, i, format(cm[i, j], fmt),horizontalalignment="center",color="white" if cm[i, j] > thresh else "black")

    plt.tight_layout()
    plt.ylabel('True label')
    plt.xlabel('Predicted label')


if __name__ == '__main__':
    name_list = ('airplane', 'automobile', 'bird', 'cat', 'deer', 'dog', 'frog', 'horse', 'ship', 'truck')
    num_classes = 10
    (x_train, y_train), (x_test, y_test) = cifar10.load_data()
    print('x_train shape:', x_train.shape)
    print(x_train.shape[0], 'train samples')
    print(x_test.shape[0], 'test samples')
    print(x_train.shape[1], 'train samples')
    print(x_test.shape[2], 'test samples')

    x_train = x_train.astype('float32')
    x_test = x_test.astype('float32')
    x_train /= 255
    x_test /= 255

    # Convert class vectors to binary class matrices.
    y_train = utils.to_categorical(y_train, num_classes)
    y_test = utils.to_categorical(y_test, num_classes)

    (_, outputdim) = y_test.shape



    #Score trained model.
    scores = model.evaluate(x_test, y_test, verbose=2)
    print('Test loss:', scores[0])
    print('Test accuracy:', scores[1])

    target = y_test
    pred = model.predict(x_test)


    ylabel = np.argmax(target,axis=1)
    yhatlabel = np.argmax(pred,axis=1)

    # Compute confusion matrix
    cnf_matrix = skl.confusion_matrix(ylabel, yhatlabel)
    np.set_printoptions(precision=2)
    is_correct = (ylabel == yhatlabel)
    acc = np.sum(is_correct * 1) / len(is_correct)
    print('accuracy:%.5f' %acc)


    # Plot non-normalized confusion matrix
    plt.figure()
    plot_confusion_matrix(cnf_matrix, classes=name_list,
                      title='Confusion matrix, without normalization')

    # Plot normalized confusion matrix
    plt.figure()
    plot_confusion_matrix(cnf_matrix, classes=name_list, normalize=True,
                      title='Normalized confusion matrix')

    plt.show()


